In [1]:
from astropy.coordinates import cartesian_to_spherical, spherical_to_cartesian

import craterdetection.common.constants as const
from craterdetection.common.camera import Camera, crater_camera_homography
from craterdetection.common.coordinates import nadir_attitude, ENU_system
from craterdetection.matching.database import extract_robbins_dataset, load_craters
from craterdetection.common.conics import *

In [2]:
lat_cat, long_cat, major_cat, minor_cat, psi_cat, crater_id = extract_robbins_dataset(
    load_craters("../data/lunar_crater_database_robbins_2018.csv", diamlims=[4, 10], ellipse_limit=1.2)
)
r_craters_cat = np.array(np.array(spherical_to_cartesian(const.RMOON, lat_cat, long_cat))).T[..., None]
C_craters_cat = crater_representation(major_cat, minor_cat, psi_cat)

In [8]:
resolution = (256, 256)
# resolution = const.CAMERA_RESOLUTION

cam_lat, cam_long = map(np.radians, (60., 75.))
cam_alt = 250

cam = Camera.from_coordinates(cam_lat, cam_long, cam_alt, resolution=resolution)

dist = cam_alt / np.cos(np.radians(cam.fov/2)*np.sqrt(2)*1.0)
vicinity = (np.sqrt(np.sum(np.square(r_craters_cat - cam.r), axis=1)) < dist).ravel()
r_craters = r_craters_cat[vicinity]
C_craters = C_craters_cat[vicinity]
A_craters = cam.project_crater_conics(C_craters, r_craters)

In [9]:
def vectorize(arr):
    return arr.reshape(-1, np.multiply(*arr.shape[-2:]), 1)
T_CM = cam.T
K = cam.K()

k = np.array([0., 0., 1.])[:, None]
S = np.concatenate((np.identity(2), np.zeros((1, 2))), axis=0)

B_craters = T_CM @ K.T @ A_craters @ K @ LA.inv(T_CM)
T_EM = np.concatenate(ENU_system(r_craters), axis=-1)
T_ME = LA.inv(T_EM)

scale_i = (vectorize(S.T @ C_craters @ S).transpose(0, 2, 1) @ vectorize(S.T @ T_ME @ B_craters @ T_EM @ S)) \
        / (vectorize(S.T @ C_craters @ S).transpose(0, 2, 1) @ vectorize(S.T @ C_craters @ S))

A = (S.T @ T_ME @ B_craters).reshape(-1, 3)
b = (S.T @ T_ME @ B_craters @ r_craters - scale_i * S.T @ C_craters @ k).reshape(-1, 1)

np.isclose(A @ cam.r, b)

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [

In [10]:
Q, R = LA.qr(A)
Qb = np.dot(Q.T, b)
LA.solve(R, Qb)

array([[ 257.14966226],
       [ 959.69560471],
       [1720.87907986]])

In [11]:
cam.r

array([[ 257.14966226],
       [ 959.69560471],
       [1720.87907986]])